Acknowledgement:
    1. The structure of the code is inspired by depmixS4: An R Package for Hidden Markov Models: https://cran.r-project.org/web/packages/depmixS4/vignettes/depmixS4.pdf
    2. Some of the linear model codes are adapted from sklearn: http://scikit-learn.org/stable/ and statsmodel: http://statsmodels.sourceforge.net/. Some modifications have been made to these codes to realize more functionalities.

Problems with existing packages
    1. Some of sklearn and statsmodels does not support the implementation of sample weights
    2. Some of sklearn and statsmodels does not support l1, l2 or elasticnet regularizations
    3. Sklearn packages does not support estimation of standard deviation of coefficients
    4. The likelihood function of weighted linear models is not the same as the ones we need to use in IO-HMM
    5. In the R package aformentioned, they do not support the provision of multiple sequences.

Modifications to above packages:
    1. Implemented supervised models that supports sample weights
    2. Supports the estimation of standard deviations of coefficients
    3. Supports multiple regularizations (l1, l2, elastic net) in most of the supervised models. (However,  if regularization is applied, no standard deviation of the coefficients will be estimated)
    4. Supports estimation over multiple sequences (multiple dataframes)
    5. HMM forward-backward code was implemented at the log scale so that it is more robust to long sequences.
    6. Supports generalized linear models with different link functions, just as statsmodel.

# Example use of UnSupervised_IOHMM

In [1]:
%load_ext autoreload
%autoreload 2

from __future__ import  division
import numpy as np
from copy import deepcopy
import sys
sys.path.append('./main')
sys.path.append('./auxiliary')
from IOHMM import UnSupervisedIOHMM, UnSupervisedIOHMMMapReduce
from SupervisedModels import LM, MNLP, MNLD
from scipy.misc import logsumexp
import pandas as pd
import warnings
warnings.simplefilter("ignore")

## Speed data - example 1

In [2]:
speed = pd.read_csv('data/speed.csv')
speed.head()

,Unnamed: 0,rt,corr,Pacc,prev
0,1,6.456770,cor,0.0,inc
1,2,5.602119,cor,0.0,cor
2,3,6.253829,inc,0.0,cor
3,4,5.451038,inc,0.0,inc
4,5,5.872118,inc,0.0,inc


## Setting up the model

In [3]:
SHMM = UnSupervisedIOHMM(num_states=2, max_EM_iter=1000, EM_tol=1e-2)
SHMM.setModels(model_emissions = [LM()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[]])
SHMM.setOutputs([['rt']])
SHMM.setData([speed])

In [4]:
SHMM.train()

-305.300566605
-305.216162094
-305.162269755
-305.084300162
-304.95789699
-304.752239908
-304.39123161
-303.600197211
-301.73925858
-295.148884087
-273.016305927
-199.390609561
-110.701760328
-93.0289858768
-92.4321980066
-92.404432368
-92.4174791498
-92.4356121371
-92.4559823475
-92.4720757993
-92.4847667687
-92.4945053257


## See the coefficients

In [5]:
print np.exp(SHMM.model_transition[0].coef - logsumexp(SHMM.model_transition[0].coef))
print np.exp(SHMM.model_transition[1].coef - logsumexp(SHMM.model_transition[1].coef))

[[ 0.91035447  0.08964553]]
[[ 0.19955373  0.80044627]]


In [6]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[1][0].coef

[ 6.38013614]
[ 5.5021878]


In [7]:
print np.sqrt(SHMM.model_emissions[0][0].dispersion)
print np.sqrt(SHMM.model_emissions[1][0].dispersion)

0.247034141013
0.182740623875


## Speed data - example 2

In [8]:
SHMM = UnSupervisedIOHMM(num_states=2, max_EM_iter=1000, EM_tol=1e-2)
SHMM.setModels(model_emissions = [LM(est_sd = True), MNLD(est_sd=True)], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[],['Pacc']])
SHMM.setOutputs([['rt'],['corr']])
SHMM.setData([speed])

In [9]:
SHMM.train()

-530.302930865
-530.239310475
-530.19737629
-530.158889512
-530.080562677
-530.01226267
-529.935550104
-529.836044223
-529.710366291
-529.55033268
-529.349383019
-529.093506793
-528.768858181
-528.334012945
-527.745305345
-526.3924036
-524.998526809
-521.857759258
-515.613507632
-504.1744234
-474.563032703
-398.004513767
-323.779208179
-302.338755714
-301.538015872
-300.449032716
-302.679593328
-302.569697692
-302.542928588
-302.543635379


In [10]:
print np.exp(SHMM.model_transition[0].coef - logsumexp(SHMM.model_transition[0].coef))
print np.exp(SHMM.model_transition[1].coef - logsumexp(SHMM.model_transition[1].coef))

[[ 0.9082254  0.0917746]]
[[ 0.19758471  0.80241529]]


In [11]:
print SHMM.model_emissions[0][0].coef
print SHMM.model_emissions[0][1].coef
print SHMM.model_emissions[1][0].coef
print SHMM.model_emissions[1][1].coef

[ 6.38524675]
[[ 0.         -1.07886847]
 [ 0.         -2.2852737 ]]
[ 5.50910848]
[[ 0.         -0.22113384]
 [ 0.          0.62337172]]


In [12]:
print SHMM.model_emissions[0][0].sd
print SHMM.model_emissions[0][1].sd
print SHMM.model_emissions[1][0].sd
print SHMM.model_emissions[1][1].sd

[ 0.01499167]
[[ 0.          0.38522395]
 [ 0.          0.81187156]]
[ 0.01397556]
[[ 0.          0.15889428]
 [ 0.          0.73585989]]


## MapReduce Version

In [13]:
sc.stop()
sc = SparkContext(appName="", pyFiles=[
    './auxiliary/HMM.py',
    './auxiliary/SupervisedModels.py',
    './auxiliary/family.py',
    './main/IOHMM.py'])

In [14]:
speed = pd.read_csv('data/speed.csv', index_col=0)
indexes = [(1,1), (2,1)]
RDD = sc.parallelize(indexes)
dfs = RDD.mapValues(lambda v: speed)

In [15]:
SHMM = UnSupervisedIOHMMMapReduce(num_states=2, max_EM_iter=100, EM_tol=1e-4)
SHMM.setModels(model_emissions = [LM()], model_transition=MNLP(solver='lbfgs'))
SHMM.setInputs(covariates_initial = [], covariates_transition = [], covariates_emissions = [[]])
SHMM.setOutputs([['rt']])
SHMM.setData(dfs)

In [16]:
SHMM.train()
print 'done'

-607.883265242
-595.478995517
-473.565132683
-256.505414778
-188.506919208
-180.14018665
-178.433063289
-177.919713522
-177.675977268
-177.57622844
-177.538977247
-177.525250495
-177.519969055
-177.517794445
-177.516827105
-177.516365078
-177.516131813
-177.516009449
-177.515943685
done


In [17]:
sc.stop()